In [6]:
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import spotipy

In [7]:
# Get the client ID and client secret from the Spotify Developer site
CLIENT_ID = ''
CLIENT_SECRET = ''
AUTH_URL = 'https://accounts.spotify.com/api/token'

In [8]:
# construct the credentials object
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Get the top 1000 tracks from query 'year:2022'
artist_name = []
track_name = []
track_popularity = []
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2022', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])

In [11]:
track_df = pd.DataFrame({'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'track_popularity' : track_popularity, 'artist_id' : artist_id})
print(track_df.shape)
track_df.head()

(1000, 5)


,artist_name,track_name,track_id,track_popularity,artist_id
0,Joji,Glimpse of Us,6xGruZOHLs39ZbVccQTuPZ,99,3MZsBdqDrRTJihTHQrO6Dq
1,Ish Kevin,My Year (2022),4eHkG6yWQ4pD0zqVkV0R5a,22,27rjDVGfZG4eorQwrZt2ZD
2,Alonestar,COOKING UP,5D87MkcNRiZ4AsVMwxRrC1,32,1kwi1Q7X4jPTkHTgayxKhS
3,Bee Gees,"Night Fever - From ""Saturday Night Fever"" Soun...",0eRHyLCekRpdkuuFu237b7,0,1LZEQNv7sE11VDY3SdxQeN
4,Bad Bunny,Me Porto Bonito,6Sq7ltF9Qa7SNFBsV5Cogx,100,4q3ewBCX7sLwd24euuV69X


In [13]:
artist_popularity = []
artist_genres = []
artist_followers = []
for a_id in track_df.artist_id:
  artist = sp.artist(a_id)
  artist_popularity.append(artist['popularity'])
  artist_genres.append(artist['genres'])
  artist_followers.append(artist['followers']['total'])

In [14]:
track_df = track_df.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)
track_df.head()

,artist_name,track_name,track_id,track_popularity,artist_id,artist_popularity,artist_genres,artist_followers
0,Joji,Glimpse of Us,6xGruZOHLs39ZbVccQTuPZ,99,3MZsBdqDrRTJihTHQrO6Dq,86,[viral pop],6481642
1,Ish Kevin,My Year (2022),4eHkG6yWQ4pD0zqVkV0R5a,22,27rjDVGfZG4eorQwrZt2ZD,23,[rwandan hip hop],7033
2,Alonestar,COOKING UP,5D87MkcNRiZ4AsVMwxRrC1,32,1kwi1Q7X4jPTkHTgayxKhS,49,[],3912
3,Bee Gees,"Night Fever - From ""Saturday Night Fever"" Soun...",0eRHyLCekRpdkuuFu237b7,0,1LZEQNv7sE11VDY3SdxQeN,75,"[disco, mellow gold, soft rock]",5846974
4,Bad Bunny,Me Porto Bonito,6Sq7ltF9Qa7SNFBsV5Cogx,100,4q3ewBCX7sLwd24euuV69X,100,"[latin, reggaeton, trap latino]",51753134


In [18]:
track_features = []
for t_id in track_df['track_id']:
  af = sp.audio_features(t_id)
  track_features.append(af)

In [19]:
print(track_features[0][0].keys())


dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])


In [20]:
tf_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [ ]:
for item in track_features:
  for feat in item:
    tf_df = tf_df.append(feat, ignore_index=True)

In [29]:
tf_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.440,0.317,-9.258,0.0531,0.8910,0.000005,0.1410,0.268,169.914,6xGruZOHLs39ZbVccQTuPZ,233456,3
1,0.521,0.683,-6.240,0.5010,0.1590,0.000000,0.1870,0.737,139.439,4eHkG6yWQ4pD0zqVkV0R5a,161808,3
2,0.544,0.792,-8.730,0.3430,0.2610,0.000000,0.2670,0.628,87.891,5D87MkcNRiZ4AsVMwxRrC1,211766,4
3,0.726,0.779,-7.057,0.0251,0.0178,0.001780,0.1040,0.841,109.133,0eRHyLCekRpdkuuFu237b7,212800,4
4,0.911,0.712,-5.105,0.0817,0.0901,0.000027,0.0933,0.425,92.005,6Sq7ltF9Qa7SNFBsV5Cogx,178567,4


In [23]:
track_df.to_csv('track_df3.csv')

In [24]:
tf_df['duration_ms'] = pd.to_numeric(tf_df['duration_ms'])
#cols_to_drop1 = ['track_id','artist_id']
cols_to_drop2 = ['key','mode','type', 'uri','track_href','analysis_url']
#track_df = track_df.drop(columns=cols_to_drop1)
tf_df = tf_df.drop(columns=cols_to_drop2)
print(track_df.info())
print(tf_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist_name        1000 non-null   object
 1   track_name         1000 non-null   object
 2   track_id           1000 non-null   object
 3   track_popularity   1000 non-null   int64 
 4   artist_id          1000 non-null   object
 5   artist_popularity  1000 non-null   int64 
 6   artist_genres      1000 non-null   object
 7   artist_followers   1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      998 non-null    float64
 1   energy            998 non-null    float64
 2   loudness          998 non-null    float64
 3   speechiness       998 no

In [25]:
track_df['artist_name'] = track_df['artist_name'].astype("string")
track_df['track_name'] = track_df['track_name'].astype("string")
track_df['track_id'] = track_df['track_id'].astype("string")
track_df['artist_id'] = track_df['artist_id'].astype("string")
tf_df['instrumentalness'] = pd.to_numeric(tf_df['instrumentalness'])
tf_df['time_signature'] = tf_df['time_signature'].astype("category")
print(track_df.info())
print(tf_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist_name        1000 non-null   string
 1   track_name         1000 non-null   string
 2   track_id           1000 non-null   string
 3   track_popularity   1000 non-null   int64 
 4   artist_id          1000 non-null   string
 5   artist_popularity  1000 non-null   int64 
 6   artist_genres      1000 non-null   object
 7   artist_followers   1000 non-null   int64 
dtypes: int64(3), object(1), string(4)
memory usage: 62.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   danceability      998 non-null    float64 
 1   energy            998 non-null    float64 
 2   loudness          998 non-null    float64 
 3   speechin

In [26]:
track_df.to_csv('tracks.csv')

In [30]:
tf_df.to_csv('tf.csv')

In [31]:
!pwd

/Users/mfaundez/mia/2022/visualizacion/proyecto_final
